# Natural Language Processing

The entire notebook is built upon [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of Jurafsky and Martin, Speech and Language Processing, 3rd edition. Feel free to read the original book as it is a good source to learn NLP.